In [2]:
import lsst.sims.maf
lsst.sims.maf.__version__

'2.6.0.sims'

In [3]:
%matplotlib inline   
import matplotlib.pyplot as plt
import numpy as np

# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

## Minon_1016 & DDF

In [4]:
# setup database connection
outDir ='DDF'
dbFile = '/home/mount/minion_1016_sqlite.db'

# establish connection to sqllite database file
opsimdb = db.OpsimDatabase(dbFile)

# specify output directory for metrics result
resultsDb = db.ResultsDb(outDir=outDir)

In [5]:
# DDF fields have a propID of 56
opsimdb.fetchFieldsFromFieldTable(propId=56)

rec.array([( 290,  6.09794381, -1.10515999),
           ( 744,  0.        , -0.79455251),
           (1427,  0.92518409, -0.47889991),
           (2412,  0.60027804, -0.08884285),
           (2786,  2.62431809,  0.04950625)], 
          dtype=[('fieldID', '<i8'), ('fieldRA', '<f8'), ('fieldDec', '<f8')])

In [15]:
# For v3 databases:
mjdcol = 'expMJD'
degrees = False # (V3 stores lon and lat in radians)

# PassMetric just pass all values
metric_pass = metrics.simpleMetrics.PassMetric(cols=['filter','fiveSigmaDepth', mjdcol, 'expDate'])
slicer = slicers.UniSlicer()
# select one of the DDF fields
sql = 'fieldID = 744'

# bundle together
ddf = metricBundles.MetricBundle(metric_pass, slicer, sql)
bg = metricBundles.MetricBundleGroup({'ddf': ddf}, opsimdb, outDir=outDir, resultsDb=resultsDb)
bg.runAll()

Querying database Summary with constraint fieldID = 744 for columns ['filter', 'fiveSigmaDepth', 'expMJD', 'expDate']
Found 24733 visits
Running:  ['ddf']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


/home/lsst/stack/miniconda3-4.3.21-10a4fa6/Linux64/sims_maf/2.6.0.sims/python/lsst/sims/maf/db/resultsDb.py:290: UserWarning: Warning! Cannot save non-conforming summary statistic.
  warnings.warn('Warning! Cannot save non-conforming summary statistic.')


In [18]:
dt = bg.bundleDict['ddf'].metricValues.data[0]

In [19]:
dt

array([('z',  22.088572,  59760.43176 ,  15589304),
       ('z',  22.037498,  59760.445023,  15590450),
       ('y',  21.921016,  59770.355327,  16446700), ...,
       ('i',  23.842873,  63215.086885, 314071506),
       ('z',  21.844793,  63224.041917, 314845221),
       ('z',  22.407117,  63224.058936, 314846692)],
      dtype=(numpy.record, [('filter', '<U256'), ('fiveSigmaDepth', '<f8'), ('expMJD', '<f8'), ('expDate', '<i8')]))

In [22]:
# save cadence to file
outF = '/home/mount/ddf_Elias'
np.savez(outF, cadence = dt)

## Now try to read back data saved to npz

In [23]:
data = np.load('/home/mount/ddf_Elias.npz')
data.files

['cadence']

In [24]:
data['cadence']

array([('z',  22.088572,  59760.43176 ,  15589304),
       ('z',  22.037498,  59760.445023,  15590450),
       ('y',  21.921016,  59770.355327,  16446700), ...,
       ('i',  23.842873,  63215.086885, 314071506),
       ('z',  21.844793,  63224.041917, 314845221),
       ('z',  22.407117,  63224.058936, 314846692)],
      dtype=[('filter', '<U256'), ('fiveSigmaDepth', '<f8'), ('expMJD', '<f8'), ('expDate', '<i8')])

In [25]:
# check number of observations
data['cadence'].shape

(24733,)